#### <font color='green'>Author: Suranjan Jena</font>
#### <font color='green'>Date: August 25, 2016</font>
# Face Recognition using neural networks :) 


# <font color='red'> Step 1: Face Detection</font>
### This is done by using Haar Cascades.
### It is a machine learning based approach where a cascade function is trained from a lot of positive and negative images. It is then used to detect objects in other images.
### It takes an image and applies about 6000 features to check if it is a face or not!!

In [2]:
#
# Import the required libraries
#
import numpy as np
from sklearn.decomposition import PCA
from sklearn.decomposition import RandomizedPCA
import cv2

In [ ]:
cols=50
rows=50

def detectFace(path):
    img = cv2.imread(path)
    cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rects = cascade.detectMultiScale(gray, 1.3, 4, cv2.cv.CV_HAAR_SCALE_IMAGE, (20,20))
    if len(rects) == 0:
        return [], img
    rects[:, 2:] += rects[:, :2]
    return rects, img

<img src="in.jpg"style="float: left; width: 20%; margin-right: 1%; margin-bottom: 0.5em;">
<img src="out.jpg"style="float: left; width: 20%; margin-right: 1%; margin-bottom: 0.5em;">

# <font color='red'> Step 2: Extract the face and crop it to the required dimension</font>

In [ ]:
def cropFace(rects, img):
    crop_img=img[rects[0][1]:rects[0][3],rects[0][0]:rects[0][2]]
    gray=cv2.cvtColor(crop_img,cv2.COLOR_BGR2GRAY)    
    gray_resized=cv2.resize(gray,(cols,rows))
    cv2.imwrite('grayCropped.png',gray_resized)
    imgArr=np.array(gray_resized)
    return imgArr

# <font color='red'>Step 3: Feature Extraction.</font>
### For feature extraction PCA was used.
### PCA is based on the fact that some dimensional variation in a data set is more important than others.
### PCA finds out the direction where the data is most spread out.
<img src="pca.png">



### For our purpose we are just focusing on the first 49 dimensions.

In [ ]:
def Pca(imgArr):
    X=np.array(imgArr)
    pca=PCA(n_components=49)
    pca.fit(X).transform(X)
    return pca

# <font color='red'> Step 4: Prepare the training and the testing data set.</font>
### For the training set I have 2145 images out of which 1001 images are my selfies, and the rest belongs to my friends 

In [ ]:
fileX=open("Train_inpImgX.txt","w")
fileY=open("Train_inpImgY.txt","w")
n=0
for i in range(1,2145):
    rects, img = detectFace("TrainPics/sample ("+str(i)+").jpg")
    if len(rects):        
        mod_imgArr=cropFace(rects, img)
        reducedDim=Pca(mod_imgArr)
        fileX.writelines( "%s " % item for item in reducedDim.explained_variance_ratio_)
        fileX.write("\n")
        
        if(i<1002):
            fileY.writelines("%s\n"%str(1))
        else:
            fileY.writelines("%s\n"%str(0))
            
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        n=n+1
fileX.close()
fileY.close()

## The number of features we have per image is now 49. So now all the 49 eigenvalues for all the 1957 images are stored onto a text file.

# <font color='red'>Step 5: Train and test using Neural networks.</font>
<img src="nn.jpeg">
### Our network has 3 layers.
### The input layer has 49 nodes, hidden layer has 15 nodes and the output layer has 2 nodes.
<img src="tanh.gif">
<img src="tanh2.png">
### The output layer uses a Softmax function because it converts raw scores into probabilities. 
<img src="softmax.png">
### A common choice of the loss function for Softmax function is the cross-entropy loss
<img src="error.png">
### Apply the back propagation formula
<img src="derv.png">

In [6]:

# import required libraries
import numpy as np

class NeuralNetwork(object):
    # Initialize the network 
    def __init__(self):
        # number of input layer nodes
        self.inputNodes = 49
        # number of hidden layer nodes
        self.hiddenNodes = 15
        # number of output layer nodes
        self.outputNodes = 2
        # gradient descent stepsize 
        self.eta = 0.001
        # number of iterations of the gradient descent each time it is called
        self.iterations = 50000
        # Initialize the parameters to random values
        np.random.seed(0)
        self.W1 = np.random.randn(self.inputNodes, self.hiddenNodes) / np.sqrt(self.inputNodes)
        self.b1 = np.zeros((1, self.hiddenNodes))
        self.W2 = np.random.randn(self.hiddenNodes, self.outputNodes) / np.sqrt(self.hiddenNodes)
        self.b2 = np.zeros((1, self.outputNodes))
        
    # batch gradient descent for minimizing the error
    def gradient_descent(self, X, y):
        # sample size
        N = len(X)
        for i in range(self.iterations):
            # forward propagation
            s1 = X.dot(self.W1) + self.b1
            a1 = np.tanh(s1)
            s2 = a1.dot(self.W2) + self.b2
            #activation fun for output is softmax
            a2 = np.exp(s2) / np.sum(np.exp(s2), axis=1, keepdims=True)
            # backpropagating the error
            delta3 = a2
            delta3[range(N), y.astype(int)] -= 1
            dLW2 = (a1.T).dot(delta3)
            dLb2 = np.sum(delta3, axis=0, keepdims=True)
            delta2 = delta3.dot(self.W2.T) * (1 - np.power(a1, 2))
            dLW1 = np.dot(X.T, delta2)
            dLb1 = np.sum(delta2, axis=0)
            # update the weights
            self.W1 = self.W1-self.eta * dLW1
            self.b1 = self.b1-self.eta * dLb1
            self.W2 = self.W2-self.eta * dLW2
            self.b2 = self.b2-self.eta * dLb2      
        return
    # evaluate the total error on the current prediction
    def curr_error(self, X, y):
        s1 = X.dot(self.W1) + self.b1
        a1 = np.tanh(s1)
        s2 = a1.dot(self.W2) + self.b2
        a2 = np.exp(s2) / np.sum(np.exp(s2), axis=1, keepdims=True)
        N = len(X)
        error = np.sum(-np.log(a2[range(N), y.astype(int)]))
        return 1./N * error

In [7]:
X_train=np.loadtxt("Train_inpX.txt")
y_train=np.loadtxt("Train_inpY.txt")
X_test=np.loadtxt("Test_inpX.txt")
y_test=np.loadtxt("Test_inpY.txt")

In [8]:
NN=NeuralNetwork()

## Error before the neural network is trained

In [9]:
NN.curr_error(X_train,y_train)* 100

71.018192614283862

## Train the network

In [7]:
NN.gradient_descent(X_train,y_train)

## Error after training the network

In [8]:
NN.curr_error(X_train,y_train)* 100

12.764148845761275

## Error on the test data

In [9]:
NN.curr_error(X_test,y_test)* 100

6.9750218826252031

In [58]:
X_test.shape

(351L, 49L)